<a href="https://colab.research.google.com/github/Cincin3112/Cross-lingual-NLI-for-Thai/blob/main/XLM_RoBERTa_base_Cross_Lingual_Thai_NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install & import

In [ ]:
#check GPU 
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  7 10:30:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModel,
    AutoModelForSequenceClassification, 
    AutoConfig,
    Trainer, 
    TrainingArguments,
)
from datasets import (
    Dataset,
    DatasetDict,
    load_dataset,
    Features, Sequence, ClassLabel, Value
)

import pandas as pd

# load model

In [ ]:
model_name = 'xlm-roberta-base'
config = AutoConfig.from_pretrained(model_name, num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [ ]:
from datasets import list_metrics, load_metric
list_metrics()

<ipython-input-7-3cf64b9e62a1>:2: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  list_metrics()


['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'mape',
 'mase',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'nist_mt',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'smape',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'Drunper/metrica_tesi',
 'Felipehonorato/my_metric',
 'GMFTBY/dailydialog_evaluate',
 'GMFTBY/dailydialogevaluate',
 'JP-SystemsX/nDCG',
 'KevinSpaghetti/accuracyk',
 'NCSOFT/harim_plus',
 'NikitaMartynov/spell-check-metric',
 'NimaBoscarino/weat',
 'Ochiroo/rouge_mn',
 'Vertaix/vendiscore',
 'Viona/infolm',
 'Vlasta/pr_auc',
 'abdusah/aradiawer',
 'abidlabs/mean_iou',
 'abidlabs/me

In [ ]:
load_metric('accuracy')

<ipython-input-8-180ef0c0d6c3>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_metric('accuracy')


Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

# load data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#for Thai-only dataset
#!gdown 1DTktDhCzaWH5ZqXxR1F5ECCw81KxalIW

#for Thai-EN dataset
#!gdown 1jN78GdA1hddO5QgCt7efRG5Hop2D_a-g

#for Thai-ZH dataset
#!gdown 1-pHk5_R4kPtbWndhb56dUNoihz4PNAja

#for Thai-EN-ZH dataset
!gdown 1IoZ9R549JYtipOqqbpF3RAU3aS4Ne4EZ

#for Thai test set
!gdown 1Bfq_RyzGTvEj4sgVi5uH_4nS7di2jwzC

Downloading...
From: https://drive.google.com/uc?id=1IoZ9R549JYtipOqqbpF3RAU3aS4Ne4EZ
To: /content/MULTItwitter_NLI_train.csv
100% 1.80M/1.80M [00:00<00:00, 76.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Bfq_RyzGTvEj4sgVi5uH_4nS7di2jwzC
To: /content/MULTItwitter_NLI_test.csv
100% 89.3k/89.3k [00:00<00:00, 95.8MB/s]


In [ ]:
#for Thai-only dataset
#df = pd.read_csv('THtwitter_NLI.csv')

#for Thai-EN dataset
#df = pd.read_csv('MULTI_th_en_NLI.csv')

#for Thai-ZH dataset
#df = pd.read_csv('MULTI_th_zh_NLI.csv')

#for Thai-EN-ZH dataset
df = pd.read_csv('MULTItwitter_NLI_train.csv')

#for Thai test set
df_test = pd.read_csv('MULTItwitter_NLI_test.csv')

In [ ]:
#แปลง label เป็น index
labels = list(set(df['label'].to_list()))
label_to_idx = {label:i for i,label in enumerate(labels)}
df['label'] = df['label'].apply(lambda x: label_to_idx[x])
df_test['label'] = df_test['label'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
  label_list[label_to_idx[label]] = label 

In [ ]:
print(label_to_idx)

{'contradiction': 0, 'neutral': 1, 'entailment': 2}


In [ ]:
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(cls_token, sep_token, pad_token, unk_token)
print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id,  tokenizer.unk_token_id)

<s> </s> <pad> <unk>
0 2 1 3


In [ ]:
#split train, dev, test
from sklearn.model_selection import train_test_split

df_train, df_dev = train_test_split(df, train_size=0.8)

len(df_train), len(df_dev), len(df_test)

(4064, 1016, 200)

# tokenize data


In [ ]:
d = DatasetDict({'train': Dataset.from_pandas(df_train), 
                 'dev': Dataset.from_pandas(df_dev),
                 'test': Dataset.from_pandas(df_test)})

d['train'] = d['train'].remove_columns(['__index_level_0__'])
d['dev'] = d['dev'].remove_columns(['__index_level_0__'])

d

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 4064
    })
    dev: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 1016
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 200
    })
})

In [ ]:
#tokenizer 
def tokenize(examples):
    tokenized_inputs = tokenizer(examples['premise'], 
                                 examples['hypothesis'], 
                                 add_special_tokens = True,
                                 truncation=True,
                                 is_split_into_words=False,
                                 max_length=250)
                                 #pad_to_max_length = True)
    return tokenized_inputs

tokenized_datasets = d.map(tokenize, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(tokenized_datasets['train'][0])

{'premise': '这种生活在地球上也几乎令人难以置信。我们的世界仍然有很多搜索。', 'hypothesis': 'เรายังรู้จักโลกไม่หมด', 'label': 2, 'input_ids': [0, 6, 7382, 2985, 213, 37986, 575, 886, 32870, 18072, 57175, 15760, 5070, 30, 15113, 3221, 29137, 27264, 63449, 30, 2, 2, 19303, 12201, 65621, 25834, 3379, 29652, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4064
    })
    dev: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1016
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})

In [ ]:
#drop column ที่ไม่ใช่ทิ้ง
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True, pad_to_multiple_of=8)

tokenized_datasets = tokenized_datasets.remove_columns(["premise"])
tokenized_datasets = tokenized_datasets.remove_columns(["hypothesis"])
#tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['label', 'input_ids', 'attention_mask']

# train model

In [ ]:
args = TrainingArguments(
    output_dir='cross-lingual-Thai-NLI', 
    learning_rate = 8e-6,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=15, 
    load_best_model_at_end=True, 
    metric_for_best_model='accuracy',
    evaluation_strategy='steps', 
    eval_steps=500, 
    #save_strategy='epoch'    
)

#metric.compute(predictions=preds, references=predictions.label_ids)
import numpy as np

def compute_metrics(eval_preds):
  metric = load_metric("accuracy")
  logits, labels = eval_preds #logists = probability แต่ละ label
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

#train model 
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4064
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3810
  Number of trainable parameters = 278045955
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,1.106900,1.099726,0.334646
1000,1.096900,1.071035,0.419291
1500,0.916200,0.855673,0.627953
2000,0.645200,0.797075,0.691929
2500,0.454500,0.737611,0.729331
3000,0.341400,0.710340,0.763780
3500,0.245200,0.708460,0.771654


***** Running Evaluation *****
  Num examples = 1016
  Batch size = 16
Saving model checkpoint to cross-lingual-Thai-NLI/checkpoint-500
Configuration saved in cross-lingual-Thai-NLI/checkpoint-500/config.json
Model weights saved in cross-lingual-Thai-NLI/checkpoint-500/pytorch_model.bin
tokenizer config file saved in cross-lingual-Thai-NLI/checkpoint-500/tokenizer_config.json
Special tokens file saved in cross-lingual-Thai-NLI/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1016
  Batch size = 16
Saving model checkpoint to cross-lingual-Thai-NLI/checkpoint-1000
Configuration saved in cross-lingual-Thai-NLI/checkpoint-1000/config.json
Model weights saved in cross-lingual-Thai-NLI/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in cross-lingual-Thai-NLI/checkpoint-1000/tokenizer_config.json
Special tokens file saved in cross-lingual-Thai-NLI/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1016
  

TrainOutput(global_step=3810, training_loss=0.6480542721085035, metrics={'train_runtime': 1532.8802, 'train_samples_per_second': 39.768, 'train_steps_per_second': 2.486, 'total_flos': 3002781879820800.0, 'train_loss': 0.6480542721085035, 'epoch': 15.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)

***** Running Prediction *****
  Num examples = 200
  Batch size = 16


In [ ]:
predicted_labels = [labels[x] for x in preds]
#gold = [labels[x] for x in predictions.label_ids]
gold = [labels[x] for x in df_test['label']]

print(predicted_labels)
print(gold)

['entailment', 'neutral', 'neutral', 'neutral', 'neutral', 'entailment', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'neutral', 'contradiction', 'entailment', 'contradiction', 'neutral', 'neutral', 'neutral', 'contradiction', 'contradiction', 'entailment', 'contradiction', 'contradiction', 'entailment', 'entailment', 'neutral', 'contradiction', 'neutral', 'neutral', 'neutral', 'neutral', 'contradiction', 'entailment', 'neutral', 'entailment', 'neutral', 'entailment', 'contradiction', 'contradiction', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'entailment', 'neutral', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'entailment', 'neutral', 'entailment', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'neutral', 'entailment', 'contradiction', 'neutral', 'neutral', 'contradiction', 'entailment', 'entailment', 'entailment', 'neutral', 'entailment', 'entailment', 'neutral', 'contradiction

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gold, predicted_labels))

               precision    recall  f1-score   support

contradiction       0.89      0.82      0.85        66
   entailment       0.74      0.85      0.79        67
      neutral       0.81      0.75      0.78        67

     accuracy                           0.81       200
    macro avg       0.81      0.81      0.81       200
 weighted avg       0.81      0.81      0.81       200

